In [1]:
# imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# google trends
# Keyword: "Auto Finanzierung"
gt_auto_finanzierung_1 = pd.read_csv('./original_data/gt_auto_finanzierung_1.csv')
gt_auto_finanzierung_2 = pd.read_csv('./original_data/gt_auto_finanzierung_2.csv')
gt_auto_finanzierung_3 = pd.read_csv('./original_data/gt_auto_finanzierung_3.csv')
# Keyword: "Auto Leasing"
gt_auto_leasing_1 = pd.read_csv('./original_data/gt_auto_leasing_1.csv')
gt_auto_leasing_2 = pd.read_csv('./original_data/gt_auto_leasing_2.csv')
gt_auto_leasing_3 = pd.read_csv('./original_data/gt_auto_leasing_3.csv')
# Keyword: "BMW Finanzierung"
gt_bmw_finanzierung_1 = pd.read_csv('./original_data/gt_bmw_finanzierung_1.csv')
gt_bmw_finanzierung_2 = pd.read_csv('./original_data/gt_bmw_finanzierung_2.csv')
gt_bmw_finanzierung_3 = pd.read_csv('./original_data/gt_bmw_finanzierung_3.csv')
# Keyword: "BMW Leasing"
gt_bmw_leasing_1 = pd.read_csv('./original_data/gt_bmw_leasing_1.csv')
gt_bmw_leasing_2 = pd.read_csv('./original_data/gt_bmw_leasing_2.csv')
gt_bmw_leasing_3 = pd.read_csv('./original_data/gt_bmw_leasing_3.csv')
# Keyword: "Leasing"
gt_leasing_1 = pd.read_csv('./original_data/gt_leasing_1.csv')
gt_leasing_2 = pd.read_csv('./original_data/gt_leasing_2.csv')
gt_leasing_3 = pd.read_csv('./original_data/gt_leasing_3.csv')
# Keyword: "BMW"
gt_bmw_1 = pd.read_csv('./original_data/gt_bmw_1.csv')
gt_bmw_2 = pd.read_csv('./original_data/gt_bmw_2.csv')
gt_bmw_3 = pd.read_csv('./original_data/gt_bmw_3.csv')

In [3]:
# apgi
apgi = pd.read_csv('./original_data/apgi.csv')

apgi['DATE'] = pd.to_datetime(apgi['DATE'], format='%Y-%m') + pd.offsets.MonthEnd(0)
apgi['DATE'] = apgi['DATE'].dt.strftime('%Y-%m-%d')

In [7]:
# arbeitslosenzahlen
arbeitslosenquote = pd.read_csv("./original_data/arbeitslosenzahlen.csv", sep=";", decimal=",")

# Clean integer columns (remove dots, convert to int)
for col in ["Bestand Arbeitslose", "Zugang Arbeitslose", "Abgang Arbeitslose"]:
  arbeitslosenquote[col] = arbeitslosenquote[col].astype(str).str.replace(".", "", regex=False).replace("", "0").astype(int)

# Clean float column (replace comma with dot, convert to float)
arbeitslosenquote["Arbeitslosenquote"] = arbeitslosenquote["Arbeitslosenquote"].astype(str).str.replace(",", ".", regex=False).replace("", "0").astype(float)

# Convert 'Berichtsmonat' to datetime
arbeitslosenquote["DATE"] = arbeitslosenquote["Berichtsmonat"].astype(str).apply(
  lambda x: pd.to_datetime(
    f"{x.split(' ')[1]}-{month_map.get(x.split(' ')[0], '01')}-01", errors="coerce"
  )
)
arbeitslosenquote["DATE"] = arbeitslosenquote["DATE"] + pd.offsets.MonthEnd(0)

# Reorder columns
arbeitslosenquote = arbeitslosenquote.drop(columns=["Berichtsmonat"])
arbeitslosenquote = arbeitslosenquote[["DATE"] + [col for col in arbeitslosenquote.columns if col != "DATE"]]

arbeitslosenquote


,DATE,Bestand Arbeitslose,Zugang Arbeitslose,Abgang Arbeitslose,Arbeitslosenquote
0,2025-04-30,2931550,562010,597530,6.3
1,2025-03-31,2967080,577380,599530,6.4
2,2025-02-28,2989220,585600,589050,6.4
3,2025-01-31,2992660,625980,439960,6.4
4,2024-12-31,2806630,562260,529670,6.0
...,...,...,...,...,...
211,2007-09-30,3530640,673130,834200,8.4
212,2007-08-31,3691890,671520,680410,8.8
213,2007-07-31,3701000,775270,746800,8.8
214,2007-06-30,3672930,578000,700850,8.8


In [ ]:
# euribor
# 3 months
euribor_3m = pd.read_csv('./original_data/euribor_3m.csv')
euribor_3m = euribor_3m.drop(columns=['TIME PERIOD'])
euribor_3m = euribor_3m.rename(
    columns={
        "Euribor 3-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA)": "Euribor 3M"
    }
)
# 6 months
euribor_6m = pd.read_csv('./original_data/euribor_6m.csv')
euribor_6m = euribor_6m.drop(columns=['TIME PERIOD'])
euribor_6m = euribor_6m.rename(
    columns={
        "Euribor 6-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR6MD_.HSTA)": "Euribor 6M"
    }
)
# 12 months
euribor_12m = pd.read_csv('./original_data/euribor_12m.csv')
euribor_12m = euribor_12m.drop(columns=['TIME PERIOD'])
euribor_12m = euribor_12m.rename(
    columns={
        "Euribor 1-year - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR1YD_.HSTA)": "Euribor 12M"
    }
)


,DATE,Euribor 12M
0,1994-01-31,6.3400
1,1994-02-28,6.4200
2,1994-03-31,6.5500
3,1994-04-30,6.4600
4,1994-05-31,6.2500
...,...,...
371,2024-12-31,2.4383
372,2025-01-31,2.5221
373,2025-02-28,2.4069
374,2025-03-31,2.3984


In [21]:
# gebrauchtwagenpreiseindex

gebrauchtwagenpreisindex = pd.read_csv(
    "./original_data/gebrauchtwagenpreisindex.csv"
)
gebrauchtwagenpreisindex["DATE"] = pd.to_datetime(
    gebrauchtwagenpreisindex["DATE"], errors="coerce"
) + pd.offsets.MonthEnd(0)
gebrauchtwagenpreisindex["DATE"] = gebrauchtwagenpreisindex["DATE"].dt.strftime(
    "%Y-%m-%d"
)

gebrauchtwagenpreisindex

,DATE,Gebrauchtwagenpreisindex
0,2015-01-31,100.0
1,2015-02-28,108.4
2,2015-03-31,111.4
3,2015-04-30,112.2
4,2015-05-31,109.7
...,...,...
119,2024-12-31,137.3
120,2025-01-31,135.2
121,2025-02-28,137.0
122,2025-03-31,138.8


In [26]:
# ifo beschäftigungsbarometer
beschaeftigungsbarometer = pd.read_csv(
    "./original_data/ifo_beschaeftigungsbarometer.csv"
)
beschaeftigungsbarometer["DATE"] = pd.to_datetime(
    beschaeftigungsbarometer["DATE"], errors="coerce"
) + pd.offsets.MonthEnd(0)
beschaeftigungsbarometer["DATE"] = beschaeftigungsbarometer["DATE"].dt.strftime(
    "%Y-%m-%d"
)

beschaeftigungsbarometer 

,DATE,Beschäftigungsbarometer,Verarbeitendes Gewerbe,Bauhauptgewerbe,Handel,Dienstleistungssektor
0,2002-01-31,91.9,-23.7,-45.4,-21.6,2.0
1,2002-02-28,92.1,-21.5,-37.2,-22.9,1.5
2,2002-03-31,92.5,-22.1,-34.4,-19.9,1.9
3,2002-04-30,92.0,-19.4,-38.3,-22.9,0.1
4,2002-05-31,92.1,-20.2,-36.8,-22.6,0.8
...,...,...,...,...,...,...
275,2024-12-31,92.5,-22.4,-2.0,-16.7,-2.4
276,2025-01-31,93.4,-22.3,-3.7,-15.6,1.0
277,2025-02-28,93.0,-19.6,-4.5,-15.9,-1.4
278,2025-03-31,92.8,-20.8,-1.8,-12.9,-2.8


In [44]:
# ifo geschäftsklima

In [45]:
# bmw absatzzahlen